In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("advanced_ecommerce_analytics.csv")

In [3]:
print(df.columns)

Index(['order_id', 'date', 'customer_id', 'age', 'gender', 'city', 'region',
       'income', 'category', 'subcategory', 'price', 'quantity',
       'discount_percent', 'final_amount', 'payment_method',
       'customer_segment', 'marketing_campaign', 'marketing_channel',
       'device_type', 'shipping_method', 'satisfaction_rating', 'returned',
       'acquisition_source', 'customer_lifetime_value', 'retention_score',
       'month', 'year', 'day_of_week', 'quarter', 'season',
       'registration_date', 'days_since_registration'],
      dtype='object')


In [4]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_limit) | (df[col] > upper_limit)]
    print(f"{col}: {len(outliers)} outliers")

age: 0 outliers
income: 580 outliers
price: 983 outliers
quantity: 1033 outliers
discount_percent: 0 outliers
final_amount: 1235 outliers
satisfaction_rating: 0 outliers
customer_lifetime_value: 1506 outliers
retention_score: 0 outliers
month: 0 outliers
year: 0 outliers
days_since_registration: 0 outliers


In [5]:
df['income'].sort_values(ascending=False).head(10)

12637    18624
8325     18624
9712     18624
1990     18624
14213    18624
10074    18624
4711     18177
3797     18129
9218     18129
9899     18129
Name: income, dtype: int64

In [6]:
df['price'].sort_values(ascending=False).head(20)

4893     12000
6664     11952
7153     11934
3069     11922
11207    11908
2272     11899
12949    11880
11373    11854
7194     11761
5561     11746
10340    11734
4127     11716
3078     11646
6103     11595
4143     11592
5369     11412
13466    11374
4459     11332
10287    11305
9155     11277
Name: price, dtype: int64

In [7]:
df['quantity'].sort_values(ascending=False).head(20)

47       8
0        8
1        8
13132    8
7300     8
7167     8
7152     8
7120     8
13141    8
1709     8
1715     8
13171    8
13285    8
1526     8
1791     8
1792     8
13066    8
1761     8
13099    8
13120    8
Name: quantity, dtype: int64

In [8]:

df = df.drop_duplicates()


df = df.replace(["-", "--", "NA", "N/A", "", "null"], pd.NA)


required_cols = ['price', 'quantity', 'final_amount']
df = df.dropna(subset=required_cols)



In [9]:
numeric_cols = [
    'age', 'income', 'price', 'quantity', 'discount_percent',
    'final_amount', 'customer_lifetime_value', 'retention_score',
    'days_since_registration'
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=numeric_cols)

In [10]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [11]:
df['Average Order Value'] = df['final_amount'] / df['quantity']
df['revenue_per_customer'] = df['final_amount'] / df['customer_id'].nunique()
df['discount_amount'] = df['price'] * df['discount_percent'] / 100
df['gross_revenue'] = df['price'] * df['quantity']
df['net_revenue'] = df['final_amount']
df['roi'] = (df['net_revenue'] - df['discount_amount']) / df['discount_amount']

In [12]:
orders_per_customer = df.groupby('customer_id').size()
df['conversion_rate'] = df['customer_id'].map(orders_per_customer)

In [13]:
total_customers = df['customer_id'].nunique()
total_orders = len(df)
conversion_rate = total_orders / total_customers

cr_channel = df.groupby('marketing_channel')['customer_id'].nunique() / df['customer_id'].nunique()
cr_campaign = df.groupby('marketing_campaign')['customer_id'].nunique() / df['customer_id'].nunique()

print(f"Overall Conversion Rate: {conversion_rate:.2f}")
print("\nBy Channel:\n", cr_channel)
print("\nBy Campaign:\n", cr_campaign)

Overall Conversion Rate: 5.03

By Channel:
 marketing_channel
Affiliate        0.337022
Direct           0.357478
Email            0.357143
Influencer       0.345406
Mobile App       0.338028
Outdoor          0.341717
Print            0.345741
Radio            0.343394
Referral         0.340376
Search Engine    0.345741
Social Media     0.357478
TV               0.334675
Name: customer_id, dtype: float64

By Campaign:
 marketing_campaign
Back to School     0.267270
Black Friday       0.260563
Flash Sale         0.594232
Holiday Special    0.263581
Mother Day         0.125419
New Year           0.210932
No Campaign        0.847082
Ramadan            0.118042
Spring Sale        0.218310
Summer Sale        0.359490
Valentine          0.141516
Name: customer_id, dtype: float64


In [14]:
kpi_category = df.groupby('category').agg({
    'gross_revenue': 'sum',
    'net_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum'
})

kpi_category['avg_order_value'] = kpi_category['net_revenue'] / kpi_category['quantity']
kpi_category['roi'] = (kpi_category['net_revenue'] - kpi_category['discount_amount']) / kpi_category['discount_amount']
kpi_category

,gross_revenue,net_revenue,discount_amount,quantity,avg_order_value,roi
category,,,,,,
Automotive,2039448,1530003.46,144145.89,1829,836.524582,9.614270
Beauty,1810279,1335021.14,62614.67,4341,307.537696,20.321220
Books,1265900,938979.73,76916.53,4599,204.170413,11.207775
Clothing,1765321,1311811.95,129928.62,2512,522.218133,9.096405
Electronics,11806831,8638149.71,936829.73,2571,3359.840416,8.220619
Food,692925,502354.14,25113.98,4305,116.690857,19.002968
Health,5485174,3937774.27,185255.21,10078,390.729735,20.255943
Home & Garden,6684079,5065566.05,512821.29,4343,1166.374868,8.877839
Software,1328595,1012136.95,75137.23,1739,582.022398,12.470512


In [15]:
kpi_campaign = df.groupby('marketing_campaign').agg({
    'net_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum',
    'customer_id': 'nunique'
})

kpi_campaign['revenue_per_customer'] = kpi_campaign['net_revenue'] / kpi_campaign['customer_id']
kpi_campaign['roi'] = (kpi_campaign['net_revenue'] - kpi_campaign['discount_amount']) / kpi_campaign['discount_amount']
kpi_campaign

,net_revenue,discount_amount,quantity,customer_id,revenue_per_customer,roi
marketing_campaign,,,,,,
Back to School,1671510.59,279046.41,2601,797,2097.252936,4.990081
Black Friday,1666242.21,284735.97,2549,777,2144.455869,4.851885
Flash Sale,4782722.97,731577.09,7541,1772,2699.053595,5.537552
Holiday Special,1802094.53,278911.23,2557,786,2292.741132,5.461176
Mother Day,660085.89,107213.82,1143,374,1764.935535,5.156724
New Year,1164604.16,190220.52,1975,629,1851.516948,5.122390
No Campaign,12030907.00,0.00,15314,2526,4762.829375,inf
Ramadan,574272.37,86504.31,1043,352,1631.455597,5.638656
Spring Sale,1324541.84,187474.14,2131,651,2034.626482,6.065198


In [16]:
kpi_channel = df.groupby('marketing_channel').agg({
    'net_revenue': 'sum',
    'gross_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum',
    'customer_id': 'nunique'
})

kpi_channel['avg_order_value'] = kpi_channel['net_revenue'] / kpi_channel['quantity']
kpi_channel['revenue_per_customer'] = kpi_channel['net_revenue'] / kpi_channel['customer_id']
kpi_channel['roi'] = (kpi_channel['net_revenue'] - kpi_channel['discount_amount']) / kpi_channel['discount_amount']
kpi_channel

,net_revenue,gross_revenue,discount_amount,quantity,customer_id,avg_order_value,revenue_per_customer,roi
marketing_channel,,,,,,,,
Affiliate,2269248.63,3244110,230290.28,3386,1005,670.185656,2257.958836,8.853862
Direct,2558265.78,3449843,231853.97,3636,1066,703.593449,2399.874090,10.033953
Email,2664421.06,3423988,220695.02,3435,1065,775.668431,2501.803812,11.072864
Influencer,2355706.59,3230306,213453.66,3591,1030,656.002949,2287.093777,10.036150
Mobile App,2314844.33,3225851,229446.62,3382,1008,684.460180,2296.472550,9.088814
Outdoor,2464162.33,3278516,228207.61,3523,1019,699.449994,2418.216222,9.797897
Print,2424964.46,3274121,212750.49,3471,1031,698.635684,2352.050883,10.398162
Radio,2331049.15,3114629,199560.01,3393,1024,687.017138,2276.415186,10.680943
Referral,2346574.16,3036091,206252.94,3321,1015,706.586618,2311.895724,10.377167


In [17]:
kpi_segment = df.groupby('customer_segment').agg({
    'net_revenue': 'sum',
    'gross_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum',
    'customer_id': 'nunique',
    'customer_lifetime_value': 'mean',
    'retention_score': 'mean'
})

kpi_segment['avg_order_value'] = kpi_segment['net_revenue'] / kpi_segment['quantity']
kpi_segment['revenue_per_customer'] = kpi_segment['net_revenue'] / kpi_segment['customer_id']
kpi_segment['roi'] = (kpi_segment['net_revenue'] - kpi_segment['discount_amount']) / kpi_segment['discount_amount']
kpi_segment

,net_revenue,gross_revenue,discount_amount,quantity,customer_id,customer_lifetime_value,retention_score,avg_order_value,revenue_per_customer,roi
customer_segment,,,,,,,,,,
Budget,8391757.20,11363975,738031.71,11406,807,63311.077559,65.542011,735.731825,10398.707807,10.370456
First-time,1732144.93,2274716,145080.30,2201,159,68722.737069,65.907635,786.980886,10893.993270,10.939215
Premium,6363613.64,8712489,673127.78,9874,719,54352.233058,74.910238,644.481835,8850.644840,8.453797
Regular,10756159.12,14572605,911959.37,15890,1132,58659.532380,65.513251,676.913727,9501.907350,10.794560
VIP,1601617.80,2114815,162547.48,2374,165,59313.675491,74.586982,674.649452,9706.774545,8.853231


In [18]:
kpi_region = df.groupby('region').agg({
    'net_revenue': 'sum',
    'gross_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum',
    'customer_id': 'nunique'
})

kpi_region['avg_order_value'] = kpi_region['net_revenue'] / kpi_region['quantity']
kpi_region['revenue_per_customer'] = kpi_region['net_revenue'] / kpi_region['customer_id']
kpi_region['roi'] = (kpi_region['net_revenue'] - kpi_region['discount_amount']) / kpi_region['discount_amount']
kpi_region

,net_revenue,gross_revenue,discount_amount,quantity,customer_id,avg_order_value,revenue_per_customer,roi
region,,,,,,,,
Alexandria,1992884.21,2627756,186585.62,2844,196,700.732845,10167.776582,9.680803
Delta,7391800.94,10246487,672609.36,11092,809,666.408307,9136.960371,9.989738
Greater Cairo,3884641.17,5333325,384401.36,5630,406,689.989551,9568.081700,9.105690
Red Sea,2048853.61,2851879,194274.51,2854,202,717.888441,10142.839653,9.546178
Suez Canal,5316849.27,7039437,469387.00,7622,553,697.566160,9614.555642,10.327219
Upper Egypt,8210263.49,10939716,723488.79,11703,816,701.552037,10061.597414,10.348156


In [19]:
kpi_month = df.groupby('month').agg({
    'net_revenue': 'sum',
    'gross_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum'
})

kpi_month['avg_order_value'] = kpi_month['net_revenue'] / kpi_month['quantity']
kpi_month['roi'] = (kpi_month['net_revenue'] - kpi_month['discount_amount']) / kpi_month['discount_amount']
kpi_month

,net_revenue,gross_revenue,discount_amount,quantity,avg_order_value,roi
month,,,,,,
1,2543529.16,3509760,190220.52,3792,670.761909,12.371476
2,1836067.11,2352447,107185.91,2763,664.519403,16.129743
3,2161145.98,2945309,205149.99,3176,680.461581,9.534468
4,2050820.12,2668642,176042.28,3184,644.101796,10.649588
5,2103802.17,2807709,154864.17,3342,629.503941,12.584822
6,2365979.04,3204784,227543.51,3103,762.481160,9.397919
7,2470166.03,3406105,249162.65,3526,700.557581,8.913870
8,2552888.22,3410743,237167.69,3730,684.420434,9.764064
9,2451771.94,3324742,250467.22,3635,674.490217,8.788794


In [20]:
kpi_quarter = df.groupby('quarter').agg({
    'net_revenue': 'sum',
    'gross_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum'
})

kpi_quarter['avg_order_value'] = kpi_quarter['net_revenue'] / kpi_quarter['quantity']
kpi_quarter['roi'] = (kpi_quarter['net_revenue'] - kpi_quarter['discount_amount']) / kpi_quarter['discount_amount']
kpi_quarter

,net_revenue,gross_revenue,discount_amount,quantity,avg_order_value,roi
quarter,,,,,,
Q1,6540742.25,8807516,502556.42,9731,672.155200,12.014941
Q2,6520601.33,8681135,558449.96,9629,677.183646,10.676250
Q3,7474826.19,10141590,736797.56,10891,686.330566,9.145020
Q4,8309122.92,11408359,832942.70,11494,722.909598,8.975624


In [21]:
kpi_season = df.groupby('season').agg({
    'net_revenue': 'sum',
    'gross_revenue': 'sum',
    'discount_amount': 'sum',
    'quantity': 'sum'
})

kpi_season['avg_order_value'] = kpi_season['net_revenue'] / kpi_season['quantity']
kpi_season['roi'] = (kpi_season['net_revenue'] - kpi_season['discount_amount']) / kpi_season['discount_amount']
kpi_season

,net_revenue,gross_revenue,discount_amount,quantity,avg_order_value,roi
season,,,,,,
Fall,7806391.13,10852741,807550.34,11294,691.198081,8.666755
Spring,6315768.27,8421660,536056.44,9702,650.975909,10.781909
Summer,7389033.29,10021632,713873.85,10359,713.296003,9.350615
Winter,7334100.00,9742567,573266.01,10390,705.880654,11.793537


In [22]:
df.to_csv("cleaned_data.csv", index=False)

In [23]:
df.dtypes

order_id                           object
date                       datetime64[ns]
customer_id                        object
age                                 int64
gender                             object
city                               object
region                             object
income                              int64
category                           object
subcategory                        object
price                               int64
quantity                            int64
discount_percent                    int64
final_amount                      float64
payment_method                     object
customer_segment                   object
marketing_campaign                 object
marketing_channel                  object
device_type                        object
shipping_method                    object
satisfaction_rating               float64
returned                             bool
acquisition_source                 object
customer_lifetime_value           

In [24]:
channel_perf = df.groupby('marketing_channel').agg({
    'discount_percent': 'sum',
    'final_amount': 'sum',
    'customer_id': 'nunique',
})

channel_perf['total_spend'] = channel_perf['discount_percent']
channel_perf['total_revenue'] = channel_perf['final_amount']
channel_perf['total_conversions'] = channel_perf['customer_id']

channel_perf['avg_roi'] = (
    (channel_perf['total_revenue'] - channel_perf['total_spend'])
    / channel_perf['total_spend']
)

channel_perf

,discount_percent,final_amount,customer_id,total_spend,total_revenue,total_conversions,avg_roi
marketing_channel,,,,,,,
Affiliate,18454,2269248.63,1005,18454,2269248.63,1005,121.967846
Direct,19704,2558265.78,1066,19704,2558265.78,1066,128.834845
Email,18810,2664421.06,1065,18810,2664421.06,1065,140.649179
Influencer,19064,2355706.59,1030,19064,2355706.59,1030,122.568327
Mobile App,18494,2314844.33,1008,18494,2314844.33,1008,124.167315
Outdoor,20320,2464162.33,1019,20320,2464162.33,1019,120.267831
Print,18910,2424964.46,1031,18910,2424964.46,1031,127.237148
Radio,18661,2331049.15,1024,18661,2331049.15,1024,123.915554
Referral,18739,2346574.16,1015,18739,2346574.16,1015,124.224087


In [25]:
best_revenue = channel_perf['total_revenue'].idxmax()
best_roi = channel_perf['avg_roi'].idxmax()
best_conversions = channel_perf['total_conversions'].idxmax()

print(f"Highest Revenue: {best_revenue}")
print(f"Best ROI: {best_roi}")
print(f"Most Conversions: {best_conversions}")

Highest Revenue: Email
Best ROI: Email
Most Conversions: Direct


In [26]:
channel_perf_sorted = channel_perf.sort_values(by='avg_roi', ascending=True)

light_layout = dict(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_x=0.5,
    yaxis=dict(showgrid=True, gridcolor="#3647F5", zeroline=False, showline=False, tickfont=dict(color="black")),
    xaxis=dict(showgrid=True, gridcolor="#3647F5", showline=False, tickfont=dict(color="black"))
)

bar_width = 25 

fig_rev = go.Figure()
fig_rev.add_trace(go.Scatter(
    x=channel_perf_sorted.index,
    y=channel_perf_sorted['total_revenue'],
    mode='markers+text',
    marker=dict(color="#3647F5", size=bar_width),
    text=channel_perf_sorted['total_revenue'],
    textposition='top center',
    texttemplate='%{text:.2s}'
))

for x_val, y_val in zip(channel_perf_sorted.index, channel_perf_sorted['total_revenue']):
    fig_rev.add_shape(
        type="line", x0=x_val, y0=0, x1=x_val, y1=y_val,
        line=dict(color="#3647F5", width=bar_width), layer="below"
    )

fig_rev.update_layout(**light_layout, title="Total Revenue per Marketing Channel", height=450, margin=dict(t=60))

fig_conv = px.scatter(
    channel_perf_sorted, x=channel_perf_sorted.index, y="total_conversions",
    size="total_conversions", color="total_conversions",
    color_continuous_scale=["#FF9F0D", "#D9D9D9"], text="total_conversions",
    title="Total Conversions per Channel"
)
fig_conv.update_traces(marker=dict(line=dict(width=2, color='#D9D9D9')), textposition='top center')
fig_conv.update_layout(**light_layout, height=450, yaxis_title="Total Conversions", xaxis_title="Marketing Channel")

fig_spend = px.line(channel_perf_sorted, x=channel_perf_sorted.index, y="total_spend", markers=True, title="Total Spend per Channel")
fig_spend.update_traces(line=dict(color="#FF9F0D", width=4), marker=dict(size=10, color="#D9D9D9", line=dict(width=2, color="#D9D9D9")))
fig_spend.update_layout(**light_layout, height=450, yaxis_title="Total Spend", xaxis_title="Marketing Channel")

fig_roi = px.bar(
    channel_perf_sorted, x='avg_roi', y=channel_perf_sorted.index, orientation='h',
    color='avg_roi', color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D'], title="Average ROI per Channel"
)
fig_roi.update_layout(**light_layout, title_font_size=22, height=450, xaxis_title="Average ROI", yaxis_title="Marketing Channel")

fig_rev.show()
fig_conv.show()
fig_spend.show()
fig_roi.show()

In [27]:
df['month_date'] = df['date'].dt.to_period('M').astype(str)

monthly_channel = df.groupby(['month_date', 'marketing_channel']).agg({
    'net_revenue': 'sum',
    'customer_id': 'nunique'
}).reset_index()

monthly_channel.columns = ['month', 'channel', 'revenue', 'conversions']
monthly_channel

,month,channel,revenue,conversions
0,2021-01,Direct,4653.92,3
1,2021-01,Influencer,4866.74,2
2,2021-01,Print,4484.62,4
3,2021-01,Radio,499.66,1
4,2021-01,Referral,773.76,1
...,...,...,...,...
433,2024-01,Radio,103830.51,58
434,2024-01,Referral,134880.22,73
435,2024-01,Search Engine,133966.94,72
436,2024-01,Social Media,122631.92,60


In [28]:
monthly_total = df.groupby('month_date').agg({
    'net_revenue': 'sum',
    'customer_id': 'nunique'
}).reset_index()
monthly_total.columns = ['month', 'total_revenue', 'total_conversions']

peak_revenue_month = monthly_total.loc[monthly_total['total_revenue'].idxmax()]
low_revenue_month = monthly_total.loc[monthly_total['total_revenue'].idxmin()]
peak_conv_month = monthly_total.loc[monthly_total['total_conversions'].idxmax()]
low_conv_month = monthly_total.loc[monthly_total['total_conversions'].idxmin()]

peaks_lows = pd.DataFrame({
    'Metric': ['Revenue', 'Revenue', 'Conversions', 'Conversions'],
    'Type': ['Peak', 'Low', 'Peak', 'Low'],
    'Month': [peak_revenue_month['month'], low_revenue_month['month'], 
              peak_conv_month['month'], low_conv_month['month']],
    'Value': [f"${peak_revenue_month['total_revenue']:,.0f}", 
              f"${low_revenue_month['total_revenue']:,.0f}",
              f"{peak_conv_month['total_conversions']:,} customers",
              f"{low_conv_month['total_conversions']:,} customers"]
})

peaks_lows

,Metric,Type,Month,Value
0,Revenue,Peak,2023-07,"$1,563,867"
1,Revenue,Low,2021-01,"$16,734"
2,Conversions,Peak,2023-07,696 customers
3,Conversions,Low,2021-01,13 customers


In [29]:
monthly_total_sorted = monthly_total.sort_values('month')
first_month_rev = monthly_total_sorted.iloc[0]['total_revenue']
last_month_rev = monthly_total_sorted.iloc[-1]['total_revenue']
revenue_growth = ((last_month_rev - first_month_rev) / first_month_rev) * 100

first_month_conv = monthly_total_sorted.iloc[0]['total_conversions']
last_month_conv = monthly_total_sorted.iloc[-1]['total_conversions']
conv_growth = ((last_month_conv - first_month_conv) / first_month_conv) * 100

avg_monthly_rev = monthly_total['total_revenue'].mean()
avg_monthly_conv = monthly_total['total_conversions'].mean()

channel_totals = monthly_channel.groupby('channel').agg({'revenue': 'sum', 'conversions': 'sum'}).reset_index()
best_channel_rev = channel_totals.loc[channel_totals['revenue'].idxmax(), 'channel']
best_channel_conv = channel_totals.loc[channel_totals['conversions'].idxmax(), 'channel']

summary = pd.DataFrame({
    'Metric': ['Revenue Growth', 'Conversions Growth', 'Avg Monthly Revenue', 'Avg Monthly Conversions', 'Best Channel (Revenue)', 'Best Channel (Conversions)'],
    'Value': [f"{revenue_growth:+,.0f}%", f"{conv_growth:+,.0f}%", f"${avg_monthly_rev:,.0f}", f"{avg_monthly_conv:,.0f}", best_channel_rev, best_channel_conv]
})

summary

,Metric,Value
0,Revenue Growth,"+8,414%"
1,Conversions Growth,"+5,046%"
2,Avg Monthly Revenue,"$779,603"
3,Avg Monthly Conversions,365
4,Best Channel (Revenue),Email
5,Best Channel (Conversions),Direct


In [30]:
from plotly.subplots import make_subplots

white_layout = dict(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_x=0.5,
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, tickfont=dict(color="black")),
    xaxis=dict(showgrid=True, gridcolor="#E0E0E0", tickangle=45, tickfont=dict(color="black"))
)

fig_rev_white = px.line(monthly_channel, x='month', y='revenue', color='channel', markers=True, title='Monthly Revenue by Channel')
fig_rev_white.update_layout(**white_layout, height=450)

fig_conv_white = px.line(monthly_channel, x='month', y='conversions', color='channel', markers=True, title='Monthly Conversions by Channel')
fig_conv_white.update_layout(**white_layout, height=450)

fig_total_rev_white = px.line(monthly_total, x='month', y='total_revenue', markers=True, title='Overall Monthly Revenue')
fig_total_rev_white.update_traces(line=dict(color="#3647F5", width=3), marker=dict(size=8, color="#FF9F0D"))
fig_total_rev_white.update_layout(**white_layout, height=400)

fig_total_conv_white = px.line(monthly_total, x='month', y='total_conversions', markers=True, title='Overall Monthly Conversions')
fig_total_conv_white.update_traces(line=dict(color="#FF9F0D", width=3), marker=dict(size=8, color="#3647F5"))
fig_total_conv_white.update_layout(**white_layout, height=400)

fig_rev_white.show()
fig_conv_white.show()
fig_total_rev_white.show()
fig_total_conv_white.show()

In [31]:
conversion_by_channel = df.groupby('marketing_channel').agg({
    'customer_id': 'nunique',
    'order_id': 'count',
}).reset_index()

conversion_by_channel.columns = ['Channel', 'Unique_Customers', 'Total_Orders']
conversion_by_channel['Conversion_Rate_%'] = (
    (conversion_by_channel['Unique_Customers'] / conversion_by_channel['Total_Orders']) * 100
).round(2)

conversion_by_channel = conversion_by_channel.sort_values('Conversion_Rate_%', ascending=False)
conversion_by_channel

,Channel,Unique_Customers,Total_Orders,Conversion_Rate_%
7,Radio,1024,1205,84.98
8,Referral,1015,1209,83.95
9,Search Engine,1031,1243,82.94
4,Mobile App,1008,1217,82.83
2,Email,1065,1288,82.69
3,Influencer,1030,1246,82.66
0,Affiliate,1005,1221,82.31
1,Direct,1066,1302,81.87
10,Social Media,1066,1302,81.87
11,TV,998,1222,81.67


In [32]:
efficiency_simple = df.groupby('marketing_channel').agg({
    'final_amount': 'sum',
    'customer_id': 'nunique',
    'order_id': 'count'
}).reset_index()

efficiency_simple.columns = ['Channel', 'Total_Revenue', 'Unique_Customers', 'Total_Orders']
efficiency_simple['Revenue_Per_Customer'] = (efficiency_simple['Total_Revenue'] / efficiency_simple['Unique_Customers']).round(2)
efficiency_simple['Orders_Per_Customer'] = (efficiency_simple['Total_Orders'] / efficiency_simple['Unique_Customers']).round(2)
efficiency_simple = efficiency_simple.sort_values('Revenue_Per_Customer', ascending=False).reset_index(drop=True)
efficiency_simple['Rank'] = range(1, len(efficiency_simple) + 1)

efficiency_simple[['Rank', 'Channel', 'Revenue_Per_Customer', 'Orders_Per_Customer', 'Total_Revenue']]

,Rank,Channel,Revenue_Per_Customer,Orders_Per_Customer,Total_Revenue
0,1,Email,2501.80,1.21,2664421.06
1,2,Outdoor,2418.22,1.25,2464162.33
2,3,Direct,2399.87,1.22,2558265.78
3,4,TV,2385.89,1.22,2381122.24
4,5,Social Media,2384.86,1.22,2542261.76
5,6,Print,2352.05,1.24,2424964.46
6,7,Referral,2311.90,1.19,2346574.16
7,8,Mobile App,2296.47,1.21,2314844.33
8,9,Influencer,2287.09,1.21,2355706.59
9,10,Radio,2276.42,1.18,2331049.15


In [33]:
revenue_analysis = df.groupby('marketing_channel').agg({
    'final_amount': 'sum',
    'order_id': 'count',
    'customer_id': 'nunique',
    'quantity': 'sum'
}).reset_index()

revenue_analysis.columns = ['Channel', 'Total_Revenue', 'Total_Orders', 'Unique_Customers', 'Total_Quantity']
revenue_analysis['Revenue_Per_Order'] = (revenue_analysis['Total_Revenue'] / revenue_analysis['Total_Orders']).round(2)
revenue_analysis['Revenue_Per_Customer'] = (revenue_analysis['Total_Revenue'] / revenue_analysis['Unique_Customers']).round(2)
revenue_analysis['Market_Share_%'] = ((revenue_analysis['Total_Revenue'] / revenue_analysis['Total_Revenue'].sum()) * 100).round(2)
revenue_analysis = revenue_analysis.sort_values('Total_Revenue', ascending=False)

revenue_analysis[['Channel', 'Total_Revenue', 'Market_Share_%', 'Revenue_Per_Order', 'Revenue_Per_Customer', 'Total_Orders']]

,Channel,Total_Revenue,Market_Share_%,Revenue_Per_Order,Revenue_Per_Customer,Total_Orders
2,Email,2664421.06,9.24,2068.65,2501.80,1288
1,Direct,2558265.78,8.87,1964.87,2399.87,1302
10,Social Media,2542261.76,8.81,1952.58,2384.86,1302
5,Outdoor,2464162.33,8.54,1938.76,2418.22,1271
6,Print,2424964.46,8.41,1903.43,2352.05,1274
11,TV,2381122.24,8.25,1948.55,2385.89,1222
3,Influencer,2355706.59,8.17,1890.62,2287.09,1246
8,Referral,2346574.16,8.14,1940.92,2311.90,1209
7,Radio,2331049.15,8.08,1934.48,2276.42,1205
4,Mobile App,2314844.33,8.03,1902.09,2296.47,1217


In [34]:
customer_value = df.groupby('marketing_channel').agg({
    'customer_id': 'nunique',
    'final_amount': 'sum',
    'order_id': 'count',
    'quantity': 'sum'
}).reset_index()

customer_value.columns = ['Channel', 'Unique_Customers', 'Total_Revenue', 'Total_Orders', 'Total_Quantity']
customer_value['Revenue_Per_Customer'] = (customer_value['Total_Revenue'] / customer_value['Unique_Customers']).round(2)
customer_value['Orders_Per_Customer'] = (customer_value['Total_Orders'] / customer_value['Unique_Customers']).round(2)
customer_value['Avg_Order_Value'] = (customer_value['Total_Revenue'] / customer_value['Total_Orders']).round(2)

max_rev_per_cust = customer_value['Revenue_Per_Customer'].max()
max_orders_per_cust = customer_value['Orders_Per_Customer'].max()

customer_value['Customer_Value_Score'] = (
    (customer_value['Revenue_Per_Customer'] / max_rev_per_cust) * 70 +
    (customer_value['Orders_Per_Customer'] / max_orders_per_cust) * 30
).round(2)

customer_value = customer_value.sort_values('Customer_Value_Score', ascending=False)
customer_value[['Channel', 'Customer_Value_Score', 'Revenue_Per_Customer', 'Orders_Per_Customer', 'Unique_Customers']]

,Channel,Customer_Value_Score,Revenue_Per_Customer,Orders_Per_Customer,Unique_Customers
2,Email,99.04,2501.80,1.21,1065
5,Outdoor,97.66,2418.22,1.25,1019
1,Direct,96.43,2399.87,1.22,1066
11,TV,96.04,2385.89,1.22,998
10,Social Media,96.01,2384.86,1.22,1066
6,Print,95.57,2352.05,1.24,1031
4,Mobile App,93.29,2296.47,1.21,1008
8,Referral,93.25,2311.90,1.19,1015
3,Influencer,93.03,2287.09,1.21,1030
0,Affiliate,92.22,2257.96,1.21,1005


In [35]:
channel_metrics = df.groupby('marketing_channel').agg({
    'final_amount': 'sum',
    'customer_id': 'nunique',
    'order_id': 'count',
    'quantity': 'sum',
    'discount_percent': 'mean',
    'price': 'mean'
}).reset_index()

channel_metrics.columns = ['Channel', 'Total_Revenue', 'Unique_Customers', 
                           'Total_Orders', 'Total_Quantity', 'Avg_Discount', 'Avg_Price']

channel_metrics['Revenue_Per_Customer'] = (channel_metrics['Total_Revenue'] / channel_metrics['Unique_Customers']).round(2)
channel_metrics['Orders_Per_Customer'] = (channel_metrics['Total_Orders'] / channel_metrics['Unique_Customers']).round(2)
channel_metrics['Revenue_Per_Order'] = (channel_metrics['Total_Revenue'] / channel_metrics['Total_Orders']).round(2)

corr_customers_revenue = channel_metrics['Unique_Customers'].corr(channel_metrics['Total_Revenue'])
corr_orders_revenue = channel_metrics['Total_Orders'].corr(channel_metrics['Total_Revenue'])
corr_quantity_revenue = channel_metrics['Total_Quantity'].corr(channel_metrics['Total_Revenue'])
corr_orders_customers = channel_metrics['Total_Orders'].corr(channel_metrics['Unique_Customers'])
corr_price_revenue = channel_metrics['Avg_Price'].corr(channel_metrics['Total_Revenue'])
corr_discount_revenue = channel_metrics['Avg_Discount'].corr(channel_metrics['Total_Revenue'])
corr_rev_per_cust_orders = channel_metrics['Revenue_Per_Customer'].corr(channel_metrics['Orders_Per_Customer'])

correlation_matrix = channel_metrics[[
    'Total_Revenue', 'Unique_Customers', 'Total_Orders', 
    'Revenue_Per_Customer', 'Orders_Per_Customer', 'Avg_Price'
]].corr()

correlation_matrix.round(3)

,Total_Revenue,Unique_Customers,Total_Orders,Revenue_Per_Customer,Orders_Per_Customer,Avg_Price
Total_Revenue,1.000,0.740,0.784,0.922,0.328,0.543
Unique_Customers,0.740,1.000,0.849,0.422,0.096,0.151
Total_Orders,0.784,0.849,1.000,0.571,0.601,0.211
Revenue_Per_Customer,0.922,0.422,0.571,1.000,0.393,0.639
Orders_Per_Customer,0.328,0.096,0.601,0.393,1.000,0.078
Avg_Price,0.543,0.151,0.211,0.639,0.078,1.000


In [36]:
conversion_by_channel = df.groupby('marketing_channel').agg({
    'customer_id': 'nunique',
    'order_id': 'count'
}).reset_index()

conversion_by_channel.columns = ['Channel', 'Unique_Customers', 'Total_Orders']
conversion_by_channel['Customer_Acquisition_Rate_%'] = (
    (conversion_by_channel['Unique_Customers'] / conversion_by_channel['Total_Orders']) * 100
).round(2)
conversion_by_channel = conversion_by_channel.sort_values('Customer_Acquisition_Rate_%', ascending=False)

fig_conv = px.bar(
    conversion_by_channel, x='Channel', y='Customer_Acquisition_Rate_%',
    title='Customer Acquisition Rate by Channel',
    color='Customer_Acquisition_Rate_%',
    color_continuous_scale=['#FF9F0D', '#D9D9D9']  
)

fig_conv.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))

fig_conv.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    height=450,
    xaxis_title="Marketing Channel",
    yaxis_title="Customer Acquisition Rate (%)",
    xaxis=dict(tickangle=45, showgrid=True, gridcolor="#E0E0E0", showline=False, tickfont=dict(color="black")),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black"))
)

fig_conv.show()

In [37]:
efficiency = df.groupby('marketing_channel').agg({
    'final_amount': ['sum', 'mean'],
    'order_id': 'count',
    'customer_id': 'nunique'
}).reset_index()

efficiency.columns = ['Channel', 'Total_Revenue', 'Avg_Order_Value', 'Total_Orders', 'Unique_Customers']
efficiency['Revenue_Per_Order'] = (efficiency['Total_Revenue'] / efficiency['Total_Orders']).round(2)
efficiency['Customer_Acq_Rate_%'] = ((efficiency['Unique_Customers'] / efficiency['Total_Orders']) * 100).round(2)

max_rev = efficiency['Revenue_Per_Order'].max()
max_cust = efficiency['Customer_Acq_Rate_%'].max()
max_order = efficiency['Avg_Order_Value'].max()

efficiency['Efficiency_Score'] = (
    (efficiency['Revenue_Per_Order'] / max_rev) * 40 +
    (efficiency['Customer_Acq_Rate_%'] / max_cust) * 30 +
    (efficiency['Avg_Order_Value'] / max_order) * 30
).round(2)

efficiency = efficiency.sort_values('Efficiency_Score')

fig_efficiency = px.bar(
    efficiency, x='Efficiency_Score', y='Channel', orientation='h',
    title='Channel Efficiency Ranking',
    color='Efficiency_Score',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)

fig_efficiency.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))

fig_efficiency.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=20,
    title_x=0.5,
    height=450,
    xaxis_title="Efficiency Score",
    yaxis_title="Marketing Channel",
    xaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color='black')),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color='black'))
)

fig_efficiency.show()

In [38]:
revenue_analysis = df.groupby('marketing_channel').agg({
    'final_amount': 'sum',
    'customer_id': 'nunique',
    'order_id': 'count'
}).reset_index()

revenue_analysis.columns = ['Channel', 'Total_Revenue', 'Unique_Customers', 'Total_Orders']
revenue_analysis['Revenue_Per_Customer'] = (revenue_analysis['Total_Revenue'] / revenue_analysis['Unique_Customers']).round(2)
revenue_analysis['Avg_Order_Value'] = (revenue_analysis['Total_Revenue'] / revenue_analysis['Total_Orders']).round(2)

max_rev = revenue_analysis['Total_Revenue'].max()
max_customers = revenue_analysis['Unique_Customers'].max()
revenue_analysis['Value_Score'] = (
    (revenue_analysis['Total_Revenue'] / max_rev) * 60 +
    (revenue_analysis['Unique_Customers'] / max_customers) * 40
).round(2)

fig_revenue = px.scatter(
    revenue_analysis, x='Unique_Customers', y='Total_Revenue',
    size='Revenue_Per_Customer', color='Value_Score',
    hover_name='Channel',
    hover_data=['Unique_Customers', 'Total_Revenue', 'Revenue_Per_Customer', 'Value_Score'],
    title='Channel Performance: Customers vs Revenue',
    size_max=60,
    color_continuous_scale=['#FF9F0D', '#3647F5', '#D9D9D9']
)

fig_revenue.update_traces(
    text=revenue_analysis['Channel'],
    textposition='top center',
    textfont=dict(size=12, color='#040D2F'),
    marker=dict(line=dict(width=2, color='#040D2F'), opacity=0.85)
)

fig_revenue.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    height=500,
    xaxis_title="Unique Customers Acquired",
    yaxis_title="Total Revenue ($)",
    xaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=True, zerolinecolor="#040D2F", showline=False, tickfont=dict(color="black")),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=True, zerolinecolor="#040D2F", showline=False, tickfont=dict(color="black")),
    showlegend=False
)

fig_revenue.show()

In [39]:
order_value_analysis = df.groupby('marketing_channel').agg({
    'final_amount': ['sum', 'mean'],
    'order_id': 'count'
}).reset_index()

order_value_analysis.columns = ['Channel', 'Total_Revenue', 'Avg_Order_Value', 'Total_Orders']
order_value_analysis = order_value_analysis.sort_values('Avg_Order_Value')

fig_order_value = px.bar(
    order_value_analysis, x='Channel', y='Avg_Order_Value',
    title='Average Order Value by Channel',
    color='Avg_Order_Value',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)

fig_order_value.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))

fig_order_value.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    height=450,
    xaxis_title="Marketing Channel",
    yaxis_title="Average Order Value ($)",
    xaxis=dict(tickangle=45, showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black")),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black"))
)

fig_order_value.show()

In [40]:
performance_by_channel = df.groupby('marketing_channel').agg({
    'final_amount': ['sum', 'mean'],
    'order_id': 'count',
    'customer_id': 'nunique'
}).reset_index()

performance_by_channel.columns = ['Channel', 'Total_Revenue', 'Avg_Order_Value', 'Total_Orders', 'Unique_Customers']
performance_by_channel['Revenue_Per_Order'] = (performance_by_channel['Total_Revenue'] / performance_by_channel['Total_Orders']).round(2)
performance_by_channel = performance_by_channel.sort_values('Revenue_Per_Order')

fig_performance = px.bar(
    performance_by_channel, x='Revenue_Per_Order', y='Channel', orientation='h',
    title='Revenue Per Order by Channel',
    color='Revenue_Per_Order',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)

fig_performance.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))

fig_performance.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    height=450,
    xaxis_title="Revenue Per Order ($)",
    yaxis_title="Marketing Channel",
    xaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black")),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black"))
)

fig_performance.show()

In [41]:
conversion_by_channel = df.groupby('marketing_channel').agg({
    'customer_id': 'nunique',
    'order_id': 'count'
}).reset_index()

conversion_by_channel.columns = ['Channel', 'Unique_Customers', 'Total_Orders']
conversion_by_channel['Customer_Acquisition_Rate_%'] = (
    (conversion_by_channel['Unique_Customers'] / conversion_by_channel['Total_Orders']) * 100
).round(2)
conversion_by_channel = conversion_by_channel.sort_values('Customer_Acquisition_Rate_%', ascending=False)

fig_conv = px.bar(
    conversion_by_channel, x='Channel', y='Customer_Acquisition_Rate_%',
    title='Customer Acquisition Rate by Channel',
    color='Customer_Acquisition_Rate_%',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)

fig_conv.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))

fig_conv.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    height=450,
    xaxis_title="Marketing Channel",
    yaxis_title="Customer Acquisition Rate (%)",
    xaxis=dict(tickangle=45, showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black")),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black"))
)

fig_conv.show()

In [42]:
efficiency = df.groupby('marketing_channel').agg({
    'final_amount': ['sum', 'mean'],
    'order_id': 'count',
    'customer_id': 'nunique'
}).reset_index()

efficiency.columns = ['Channel', 'Total_Revenue', 'Avg_Order_Value', 'Total_Orders', 'Unique_Customers']
efficiency['Revenue_Per_Order'] = (efficiency['Total_Revenue'] / efficiency['Total_Orders']).round(2)
efficiency['Customer_Acq_Rate_%'] = ((efficiency['Unique_Customers'] / efficiency['Total_Orders']) * 100).round(2)

max_rev = efficiency['Revenue_Per_Order'].max()
max_cust = efficiency['Customer_Acq_Rate_%'].max()
max_order = efficiency['Avg_Order_Value'].max()

efficiency['Efficiency_Score'] = (
    (efficiency['Revenue_Per_Order'] / max_rev) * 40 +
    (efficiency['Customer_Acq_Rate_%'] / max_cust) * 30 +
    (efficiency['Avg_Order_Value'] / max_order) * 30
).round(2)

efficiency = efficiency.sort_values('Efficiency_Score')
efficiency[['Channel', 'Efficiency_Score', 'Revenue_Per_Order', 'Customer_Acq_Rate_%', 'Avg_Order_Value']]

,Channel,Efficiency_Score,Revenue_Per_Order,Customer_Acq_Rate_%,Avg_Order_Value
9,Search Engine,88.97,1764.02,82.94,1764.016251
0,Affiliate,91.95,1858.52,82.31,1858.516486
6,Print,92.98,1903.43,80.93,1903.425793
3,Influencer,93.16,1890.62,82.66,1890.615241
4,Mobile App,93.60,1902.09,82.83,1902.090657
5,Outdoor,93.91,1938.76,80.17,1938.758718
11,TV,94.77,1948.55,81.67,1948.545205
10,Social Media,94.97,1952.58,81.87,1952.581997
8,Referral,95.31,1940.92,83.95,1940.921555
1,Direct,95.39,1964.87,81.87,1964.873871


In [43]:
quadrant_analysis = df.groupby('marketing_channel').agg({
    'customer_id': 'nunique',
    'final_amount': 'sum',
    'order_id': 'count'
}).reset_index()

quadrant_analysis.columns = ['Channel', 'Unique_Customers', 'Total_Revenue', 'Total_Orders']
quadrant_analysis['Revenue_Per_Customer'] = (quadrant_analysis['Total_Revenue'] / quadrant_analysis['Unique_Customers']).round(2)

avg_customers = quadrant_analysis['Unique_Customers'].mean()
avg_revenue = quadrant_analysis['Total_Revenue'].mean()

fig_quadrant = px.scatter(
    quadrant_analysis, x='Unique_Customers', y='Total_Revenue',
    size='Revenue_Per_Customer', color='Revenue_Per_Customer',
    hover_name='Channel', hover_data=['Revenue_Per_Customer', 'Total_Orders'],
    title='Performance Quadrant: High Revenue/High Reach = Top Right',
    size_max=50, color_continuous_scale=['#FF9F0D', '#3647F5']
)

fig_quadrant.update_traces(marker=dict(line=dict(width=2, color='#040D2F'), opacity=0.9))

fig_quadrant.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    height=500,
    xaxis_title="Unique Customers Acquired",
    yaxis_title="Total Revenue ($)",
    xaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=True, linewidth=2, linecolor="#040D2F", tickfont=dict(color="black")),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=True, linewidth=2, linecolor="#040D2F", tickfont=dict(color="black")),
    showlegend=False
)

fig_quadrant.add_hline(y=avg_revenue, line_dash="dash", line_color="#3647F5", annotation_text=f"Avg Revenue: ${avg_revenue:,.0f}", annotation_position="right")
fig_quadrant.add_vline(x=avg_customers, line_dash="dash", line_color="#3647F5", annotation_text=f"Avg Customers: {avg_customers:,.0f}", annotation_position="top")

fig_quadrant.show()

In [44]:
customer_value = df.groupby('marketing_channel').agg({
    'final_amount': 'sum',
    'customer_id': 'nunique',
    'order_id': 'count'
}).reset_index()

customer_value.columns = ['Channel', 'Total_Revenue', 'Total_Customers', 'Total_Orders']
customer_value['Revenue_Per_Customer'] = (customer_value['Total_Revenue'] / customer_value['Total_Customers']).round(2)
customer_value = customer_value.sort_values('Revenue_Per_Customer')

fig_customer_value = px.bar(
    customer_value, x='Channel', y='Revenue_Per_Customer',
    title='Revenue Per Customer by Channel',
    color='Revenue_Per_Customer',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)

fig_customer_value.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))

fig_customer_value.update_layout(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    height=450,
    xaxis_title="Marketing Channel",
    yaxis_title="Revenue Per Customer ($)",
    xaxis=dict(tickangle=45, showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black")),
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, showline=False, tickfont=dict(color="black"))
)

fig_customer_value.show()

In [45]:
best_revenue_per_customer = customer_value.sort_values('Revenue_Per_Customer', ascending=False).iloc[0]
worst_revenue_per_customer = customer_value.sort_values('Revenue_Per_Customer').iloc[0]
best_acquisition = conversion_by_channel.iloc[0]

best_efficiency = efficiency.sort_values('Efficiency_Score', ascending=False).iloc[0]
worst_efficiency = efficiency.sort_values('Efficiency_Score').iloc[0]

best_total_revenue = revenue_analysis.sort_values('Total_Revenue', ascending=False).iloc[0]
best_order_value = revenue_analysis.sort_values('Revenue_Per_Customer', ascending=False).iloc[0]

insights = pd.DataFrame({
    'Metric': [
        'Highest Revenue/Customer', 'Lowest Revenue/Customer', 'Best Customer Acquisition',
        'Most Efficient Channel', 'Least Efficient Channel',
        'Highest Total Revenue', 'Best Value/Customer'
    ],
    'Channel': [
        best_revenue_per_customer['Channel'], worst_revenue_per_customer['Channel'], best_acquisition['Channel'],
        best_efficiency['Channel'], worst_efficiency['Channel'],
        best_total_revenue['Channel'], best_order_value['Channel']
    ],
    'Value': [
        f"${best_revenue_per_customer['Revenue_Per_Customer']:,.2f}",
        f"${worst_revenue_per_customer['Revenue_Per_Customer']:,.2f}",
        f"{best_acquisition['Customer_Acquisition_Rate_%']:.2f}%",
        f"Score: {best_efficiency['Efficiency_Score']:.2f}",
        f"Score: {worst_efficiency['Efficiency_Score']:.2f}",
        f"${best_total_revenue['Total_Revenue']:,.2f}",
        f"${best_order_value['Revenue_Per_Customer']:,.2f}"
    ]
})

insights

,Metric,Channel,Value
0,Highest Revenue/Customer,Email,"$2,501.80"
1,Lowest Revenue/Customer,Search Engine,"$2,126.74"
2,Best Customer Acquisition,Radio,84.98%
3,Most Efficient Channel,Email,Score: 99.19
4,Least Efficient Channel,Search Engine,Score: 88.97
5,Highest Total Revenue,Email,"$2,664,421.06"
6,Best Value/Customer,Email,"$2,501.80"


In [46]:
light_layout = dict(
    plot_bgcolor="#FFFFFF",
    paper_bgcolor="#FFFFFF",
    font_color="#040D2F",
    title_font_size=22,
    title_x=0.5,
    yaxis=dict(showgrid=True, gridcolor="#E0E0E0", zeroline=False, tickfont=dict(color="#040D2F")),
    xaxis=dict(showgrid=True, gridcolor="#E0E0E0", tickangle=45, tickfont=dict(color="#040D2F"))
)

fig_revenue_order = px.bar(
    performance_by_channel.sort_values('Revenue_Per_Order'),
    x='Revenue_Per_Order', y='Channel', orientation='h',
    title='Revenue Per Order by Channel',
    color='Revenue_Per_Order',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)
fig_revenue_order.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))
fig_revenue_order.update_layout(**light_layout, height=450, xaxis_title="Revenue Per Order ($)", yaxis_title="Marketing Channel")

fig_acquisition = px.bar(
    conversion_by_channel.sort_values('Customer_Acquisition_Rate_%', ascending=False),
    x='Channel', y='Customer_Acquisition_Rate_%',
    title='Customer Acquisition Rate by Channel',
    color='Customer_Acquisition_Rate_%',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)
fig_acquisition.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))
fig_acquisition.update_layout(**light_layout, height=450, xaxis_title="Marketing Channel", yaxis_title="Customer Acquisition Rate (%)")

fig_efficiency_final = px.bar(
    efficiency.sort_values('Efficiency_Score'),
    x='Efficiency_Score', y='Channel', orientation='h',
    title='Channel Efficiency Ranking',
    color='Efficiency_Score',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)
fig_efficiency_final.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))
fig_efficiency_final.update_layout(**light_layout, height=450, xaxis_title="Efficiency Score", yaxis_title="Marketing Channel")

fig_revenue_customers = px.scatter(
    revenue_analysis, x='Unique_Customers', y='Total_Revenue',
    size='Revenue_Per_Customer', color='Revenue_Per_Customer', text='Channel',
    title='Revenue vs Customer Acquisition',
    color_continuous_scale=['#FF9F0D', '#3647F5', '#D9D9D9']
)
fig_revenue_customers.update_traces(textposition='top center', marker=dict(line=dict(width=2, color='#040D2F'), opacity=0.85))
fig_revenue_customers.update_layout(**light_layout, height=500, xaxis_title="Unique Customers Acquired", yaxis_title="Total Revenue ($)")

fig_revenue_customer = px.bar(
    customer_value.sort_values('Revenue_Per_Customer'),
    x='Channel', y='Revenue_Per_Customer',
    title='Revenue Per Customer by Channel',
    color='Revenue_Per_Customer',
    color_continuous_scale=['#3647F5', '#D9D9D9', '#FF9F0D']
)
fig_revenue_customer.update_traces(marker=dict(line=dict(width=1.5, color='#040D2F')))
fig_revenue_customer.update_layout(**light_layout, height=450, xaxis_title="Marketing Channel", yaxis_title="Revenue Per Customer ($)")

fig_revenue_order.show()
fig_acquisition.show()
fig_efficiency_final.show()
fig_revenue_customers.show()
fig_revenue_customer.show()